In [ ]:
import uuid
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, JSON, Table
from sqlalchemy.orm import declarative_base, relationship, sessionmaker
from sqlalchemy.dialects.postgresql import UUID

Base = declarative_base()


# Define the top-level Component model with UUID
class Component(Base):
    __tablename__ = 'components'

    id = Column(Integer, primary_key=True)
    uuid = Column(UUID(as_uuid=True), default=uuid.uuid4, unique=True, nullable=False)  # UUID field
    name = Column(String, nullable=False)
    description = Column(String, nullable=True)

    # One-to-many relationship with requirements
    requirements = relationship("Requirement", back_populates="component")

    def __repr__(self):
        return f"<Component(id={self.id}, uuid={self.uuid}, name='{self.name}')>"

# Define the Requirement model with UUID and JSON data
class Requirement(Base):
    __tablename__ = 'requirements'

    id = Column(Integer, primary_key=True)
    uuid = Column(UUID(as_uuid=True), default=uuid.uuid4, unique=True, nullable=False)  # UUID field
    component_id = Column(Integer, ForeignKey('components.id'), nullable=True)  # Made nullable for sub-requirements
    data = Column(JSON, nullable=False)  # Store Requirement as JSON

    component = relationship("Component", back_populates="requirements")
    functions = relationship("Function", secondary=requirement_function_association, back_populates="requirements")
    
    # Self-referential relationship for sub-requirements
    sub_requirements = relationship(
        "Requirement", secondary=sub_requirement_association,
        primaryjoin=id == sub_requirement_association.c.parent_id,
        secondaryjoin=id == sub_requirement_association.c.child_id,
        backref="parent_requirements"
    )

    def __repr__(self):
        return f"<Requirement(id={self.id}, uuid={self.uuid}, data={self.data})>"